# Coursework 2
### CID: 02476922

I, 02476922, certify that this assessed coursework is my own work, unless otherwise acknowledged, and includes no
plagiarism. I have not discussed my coursework with anyone else except when seeking clarification with the module
lecturer via email or on MS Teams. I have not shared any code underlying my coursework with anyone else prior to
submission.

### Question 1

In [1]:
### IMPORTS

import pandas as pd


C:\Users\frase\AppData\Local\Temp\ipykernel_26584\3934771343.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
components = pd.read_csv("q1_data/components.csv")
spectra = pd.read_csv("q1_data/spectra.csv")
temperatures = pd.read_csv("q1_data/temperatures.csv")

In [7]:
spectra

-1.107484654939132618e+00 -1.100693122333052942e+00 -1.092015052891950999e+00 -1.085475058530540826e+00 -1.081953523105166237e+00 -1.076168143477764794e+00 -1.074533144887412250e+00 -1.071263147706706942e+00 -1.073149684541729210e+00 -1.075161990499086118e+00 -1.078872179607963222e+00 -1.077488719262280181e+00 -1.079563909780804520e+00 -1.081576215738161872e+00 -1.082519484155672673e+00 -1.084909097480034168e+00 -1.084846212918866737e+00 -1.084720443796531875e+00 -1.083840059940188061e+00 -1.084657559235364888e+00 -1.083714290817853643e+00 -1.082079292227500655e+00 -1.080004101708976316e+00 -1.081073139248822423e+00 -1.080821601004152477e+00 -1.078997948730297640e+00 -1.079815448025474467e+00 -1.078872179607963222e+00 -1.080444293637148334e+00 -1.083336983450849056e+00 -1.084657559235364888e+00 -1.085097751163536461e+00 -1.086921403437391076e+00 -1.089059478517082846e+00 -1.090883130790937461e+00 -1.088493517466576410e+00 -1.089625439567589504e+00 -1.089122363078250277e+00 -1.090631592546268180e+00 -1.086544096070386489e+00 -1.085349289408205964e+00 -1.091323322719109479e+00 -1.087927556416069752e+00 -1.080444293637148334e+00 -1.081576215738161872e+00 -1.083085445206179331e+00 -1.082896791522677260e+00 -1.084028713623690354e+00 -1.079438140658469658e+00 -1.081387562054659579e+00 -1.080821601004152477e+00 -1.082393715033338033e+00 -1.082708137839174967e+00 -1.084028713623690354e+00 -1.082896791522677260e+00 -1.079941217147808885e+00 -1.073275453664064294e+00 -1.066798343863821108e+00 -1.058623350912058614e+00 -1.045857784995075201e+00 -1.037242600115140689e+00 -1.025608956299170815e+00 -1.017622617030910170e+00 -1.004291090063420322e+00 -9.917141778299394250e-01 -9.792630347187930573e-01 -9.598317053180647207e-01 -9.434188348533718571e-01 -9.217865458117843191e-01 -8.997769494031924165e-01 -8.779560066781026961e-01 -8.555062183413389976e-01 -8.292204717733634434e-01 -8.024945332772162043e-01 -7.708635990100113355e-01 -7.385409345699648354e-01 -7.082934606484428963e-01 -6.789892551444318691e-01 -6.604383096000472353e-01 -6.439625545741870516e-01 -6.315114114630407949e-01 -6.215127662374233530e-01 -6.071750862912548286e-01 -5.869891421565176515e-01 -5.601374345380355502e-01 -5.292611150048392998e-01 -4.879459583178540893e-01 -4.399021535859563015e-01 -3.861987383489919878e-01 -3.229368698145820837e-01 -2.504938553497310094e-01 -1.648450830397248656e-01 -6.573901463989378913e-02 4.487492845357238247e-02 1.767438532216216163e-01 3.274152617787250974e-01 4.946253099228561934e-01 6.812666874677156814e-01 8.677822958902402517e-01 1.057253478687632642e+00 1.235782747841897011e+00 1.413117210333980633e+00 1.591143402998905332e+00 1.768729403735658456e+00 1.946315404472411359e+00 2.098747580742201801e+00 2.215021134340735109e+00 2.256084752783050540e+00 2.223133242731330395e+00 2.111701800342687729e+00 1.953232706200825897e+00 1.755397876768168164e+00 1.568064769050467655e+00 1.388843769723361765e+00 1.236222939770068807e+00 1.116616504429663426e+00 1.027634850377785014e+00 9.672027870959082341e-01 9.178384065794950031e-01 8.861445877511227032e-01 8.611794169676627586e-01 8.452067384311418019e-01 8.248950251740697626e-01 8.134500350416018488e-01 7.970371645769090962e-01 7.839571758540887503e-01 7.696194959079202258e-01 7.584889285812894677e-01 7.486789370391742082e-01 7.371081777843716543e-01 7.305681834229614813e-01 7.192489624128285408e-01 7.074895494745236935e-01 6.951641754857122990e-01 6.799461116832000718e-01 6.613951661388154379e-01 6.410834528817436206e-01 6.103329024708823436e-01 5.807771587222017029e-01 5.425433455324191190e-01 4.994674211327464475e-01 4.515493855231835219e-01 3.997953916824088516e-01 3.467837066182860029e-01 2.932689450648239826e-01 2.303214993362510954e-01 1.768067377827890196e-01 1.179467885300975183e-01 6.669587117866208892e-02 1.921802749727098344e-02 -1.643751868464790253e-02 -4.599326243332846342e-02 -7.152439426729507999e-02 -7.341093110231727836e-02 -5.020652803154469768e-02 -1.291598325927327540e-02 4